In [0]:
%pip install -r '/Workspace/Users/chrismarshall.wi@icloud.com/dbxProjects/Alpha Factors/requirements.txt'

In [0]:
import sys 
sys.path.append('/Workspace/Users/chrismarshall.wi@icloud.com/dbxProjects/Alpha Factors')
from utilities import Market, Tickers
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.getOrCreate()

In [0]:
import pandas as pd
import requests
from io import StringIO

def get_sp500_tickers():
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }

    response = requests.get(url, headers=headers)
    response.raise_for_status()

    tables = pd.read_html(StringIO(response.text))
    df = tables[0]
    return df['Symbol'].tolist()

tickers = get_sp500_tickers()
print(len(tickers))
print(tickers[:10])


In [0]:
ticker = ['AAPL','MSFT']
tickers_all = Tickers.get_tickers_sp()
start = '2022-01-01'
end = '2025-10-25'

In [0]:
data = Market(tickers=ticker, start=start, end=end)
df = spark.createDataFrame(data=data.get_price())

In [0]:
df.write.format("delta").mode("overwrite").saveAsTable("operations.finance.fact_price_daily")